# Projet d'algorithme de trading

In [105]:
%load_ext autoreload
%load_ext sql
%autoreload 2

# importation des librairies
import requests
import pandas as pd
from datetime import date, timedelta
import sqlite3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [51]:
#url = 'https://rest.coinapi.io/v1/assets'
url = 'https://rest.coinapi.io/v1/exchangerate/BTC/USD/history?period_id=1MIN&time_start=2016-01-01T00:00:00&time_end=2016-02-01T00:00:00'
headers = {'X-CoinAPI-Key' : 'D867C7FC-B41F-45C8-B2FE-D92EF8EF3F33'}
response = requests.get(url, headers=headers)

In [58]:

asset = "USD/BTC"
start_time = (date.today() - timedelta(5)).strftime("%Y-%m-%d")
end_time = date.today().strftime("%Y-%m-%d")
        
        # Create URL


#url = 'https://rest.coinapi.io/v1/exchangerate/BTC/USD/history?period_id=1MIN&time_start=2023-03-05T00:00:00&time_end=2023-04-24T00:00:00'

url = ('https://rest.coinapi.io/v1/'
        f'exchangerate/{asset}/'
        'history?period_id=1DAY&'
        f'time_start={start_time}T00:00:00&'
        f'time_end={end_time}T00:00:00')

headers = {'X-CoinAPI-Key' : 'D867C7FC-B41F-45C8-B2FE-D92EF8EF3F33'}
print(headers)
        # send call to  api
response1 = requests.get(url, headers=headers)
        # extract json data
json_data = response1.json()
        # read data into dataframe
df = pd.DataFrame(json_data)
        #select important columns
columns = ["time_close", "rate_open", "rate_high", "rate_low", "rate_close"]
df = df[columns]
        #create new index convert it into "DatetimeIndex" and named "date" 
df = df.set_index("time_close")
df.index = pd.to_datetime(df.index)
df.index.name = "date"
        
        # remove "_rate" into columns
#df.columns = [c.split("_")[-1] for c in df.columns]

{'X-CoinAPI-Key': 'D867C7FC-B41F-45C8-B2FE-D92EF8EF3F33'}


In [59]:
response1.status_code

200

In [50]:
f'time_start={start_time}T00:00:00&'

'time_start=2023-04-17T00:00:00&'

In [46]:
start_time

'2023-04-17'

In [47]:
end_time

'2023-04-27'

In [60]:
df

,rate_open,rate_high,rate_low,rate_close
date,,,,
2023-04-22 23:59:00+00:00,0.000037,0.000037,0.000036,0.000036
2023-04-23 23:59:00+00:00,0.000036,0.000037,0.000036,0.000036
2023-04-24 23:59:00+00:00,0.000036,0.000037,0.000036,0.000036
2023-04-25 23:59:00+00:00,0.000036,0.000037,0.000035,0.000035
2023-04-26 23:59:00+00:00,0.000035,0.000037,0.000033,0.000035


In [82]:

from config import settings 
import requests
import pandas as pd
from datetime import date, timedelta
import sqlite3



class COINAPI:
    def __init__(self, api_key=settings.coin_api_key):
        self.__api_key = api_key
    def get_daily(self, asset, output_size=10):
        """ get daily time series from Coinapi

        Parameters
        ---------
        asset : str
            the ticker symbol of the asset
        output_size : int
            Number of the observations to retrieve
            maximun is 100
            
        returns
        -------
        pd.DataFrame
         columns are 'open', 'high', 'low', 'close'.
         all numeric
        """
        start_time = (date.today() - timedelta(output_size)).strftime("%Y-%m-%d")
        end_time = date.today().strftime("%Y-%m-%d")
        
        # Create URl
        url = ('https://rest.coinapi.io/v1/'
                f'exchangerate/{asset}/'
                'history?period_id=1DAY&'
                f'time_start={start_time}T00:00:00&'
                f'time_end={end_time}T00:00:00')
        
        headers = {'X-CoinAPI-Key' : f'{self.__api_key}'}
        
        # send call to  api
        response = requests.get(url, headers=headers)
        # extract json data
        json_data = response.json()
        # read data into dataframe
        df = pd.DataFrame(json_data)
        #select important columns
        columns = ["time_close", "rate_open", "rate_high", "rate_low", "rate_close"]
        df = df[columns]
        #create new index convert it into "DatetimeIndex" and named "date" 
        df = df.set_index("time_close")
        df.index = pd.to_datetime(df.index)
        df.index.name = "date"
        
        # remove "_rate" into columns
        df.columns = [c.split("_")[-1] for c in df.columns]

        #return DataFrme   
        return df

In [83]:
btc_usd = COINAPI()

In [87]:
btc_usd_data = btc_usd.get_daily(asset="BTC/USD", output_size=50)

In [88]:
btc_usd_data

,open,high,low,close
date,,,,
2023-03-09 23:59:00+00:00,21701.967560,21811.792399,20119.121002,20363.264056
2023-03-10 23:59:00+00:00,20362.344257,20362.344257,19595.033492,20231.277547
2023-03-11 23:59:00+00:00,20225.193289,20865.139865,19883.953577,20618.016385
2023-03-12 23:59:00+00:00,20621.168421,22209.315709,20466.218353,22171.980928
2023-03-13 23:59:00+00:00,22166.416360,24636.589496,21923.220403,24207.797181
2023-03-14 23:59:00+00:00,24215.760023,26495.248109,24111.172075,24764.286615
2023-03-15 23:59:00+00:00,24755.865640,25271.346521,23989.526827,24380.877973
2023-03-16 23:59:00+00:00,24377.691842,25203.447568,24220.303716,25056.279526
2023-03-17 23:59:00+00:00,25051.460791,27819.892635,24949.436515,27451.948658


In [89]:
class SQLRepository:
    def __init__(self, connection) :
        self.connection = connection
        
    def insert_table(self, table_name, records, if_exists="fail"):
        """ insert dataframe into sqlite database as table 

        Parameters
        ----------
        table_name: str
        records: pd.DataFrame
        if_exists: str, optional
        
            - 'fail': Raise a ValueError.
            - 'replace': Drop the table before inserting new values.
            - 'append': Insert new values to the existing table.

            Dafault: 'fail'
        Returns
        -------
        dict
            - 'transaction_successful', followed by bool
            - 'records_inserted', followed by int
        """
        
        n_inserted = records.to_sql(name=table_name, con=self.connection, if_exists=if_exists)
        return {"transaction_successful": True, "records_inserted": n_inserted}
    
    def read_table(self, table_name, limit=None):
        """read table from sqlite database

        Parameters
        ----------
        table_name: str
        limit: int, None, optional
            Number of most recent records to retrieve. If `None`, all
            records are retrieved. By default, `None`.

        Returns
        -------
        pd.dataFrame
            Index is DatetimeIndex "date". Columns are 'open', 'high',
            'low', 'close', and 'volume'. All columns are numeric.
        """
        pass

In [90]:
conn = sqlite3.connect('stocks')

In [91]:
conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Traceback (most recent call last):
  File "C:\Users\yoda\anaconda3\lib\site-packages\sql\connection.py", line 45, in __init__
    engine = sqlalchemy.create_engine(
  File "<string>", line 2, in create_engine
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\util\deprecations.py", line 298, in warned
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\engine\create.py", line 522, in create_engine
    :ref:`sql_caching`
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\engine\url.py", line 754, in _get_entrypoint
    cls = registry.load(name)
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\util\langhelpers.py", line 358, in load
    loader = self.auto_fn(name)
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\dialects\__init__.py", line 47, in _auto_fn
    module = __import__("sqlalchemy.dialects.%s" % (dialect,)).dialects
  File "C:\Users\yoda\anaconda3\lib\s

In [96]:
btc_usd_data.to_sql('btc_usd', conn, if_exists='replace')

In [108]:
%load_ext sql
%sql sqlite:////C:\Users\yoda\BitcoinAnalyseur/stocks

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Traceback (most recent call last):
  File "C:\Users\yoda\anaconda3\lib\site-packages\sql\connection.py", line 45, in __init__
    engine = sqlalchemy.create_engine(
  File "<string>", line 2, in create_engine
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\util\deprecations.py", line 298, in warned
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\engine\create.py", line 522, in create_engine
    :ref:`sql_caching`
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\engine\url.py", line 754, in _get_entrypoint
    cls = registry.load(name)
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\util\langhelpers.py", line 358, in load
    loader = self.auto_fn(name)
  File "C:\Users\yoda\anaconda3\lib\site-packages\sqlalchemy\dialects\__init__.py", line 47, in _auto_fn
    module = __import__("sqlalchemy.dialects.%s" % (dialect,)).dialects
  File "C:\Users\yoda\anaconda3\lib\s

In [ ]:
C:\Users\yoda\BitcoinAnalyseur